In [33]:
import configparser
import os

#import logging
#import boto3
#from botocore.exceptions import ClientError

#import pandas as pd

from pyspark.sql import SparkSession
from pyspark import SparkContext

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType

from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import functions as f

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the Configuration file

In [4]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

## Read the AWS User Access Key ID and Secret Key [Check Before executing the script]

In [7]:
os.environ['AWS_ACCESS_KEY_ID']= #config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']= #config['AWS']['AWS_SECRET_ACCESS_KEY']

KEY= #config['AWS']['AWS_ACCESS_KEY_ID']
SECRET= #config['AWS']['AWS_SECRET_ACCESS_KEY']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create a Spark Session in EMR cluster

In [8]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark = create_spark_session()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the Song data JSON file stored in S3 bucket

In [13]:
INPUT_DATA = 's3a://udacity-dend/song_data' #config['S3']['SONG_DATA']

song_data = INPUT_DATA + "/A/A/A/*.json"
song_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://udacity-dend/song_data/A/A/A/*.json'

## Read the JSON file into a Spark DataFrame

In [14]:
# read song data file
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# extract columns to create songs table
songs_table = df.select(["song_id", "title", "artist_id", "year", "duration"]).dropDuplicates()
songs_table.limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SODZYPO12A8C13A91E|Burn My Body (Alb...|AR1C2IX1187B99BF74|   0|177.99791|
|SOIGHOD12A8C13B5A1|        Indian Angel|ARY589G1187B9A9F4E|2004|171.57179|
|SOOVHYF12A8C134892|     I'll Be Waiting|ARCLYBR1187FB53913|1989|304.56118|
|SOAPERH12A58A787DC|The One And Only ...|ARZ5H0P1187B98A1DD|   0|230.42567|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
+------------------+--------------------+------------------+----+---------+

In [21]:
# Save in Parquet format in S3 bucket
songs_table.write.mode('overwrite').partitionBy('year', 'artist_id').parquet('s3a://desh1gvrk/parquet/songs')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# extract columns to create artists table
artists_table = df.select(["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet('s3a://desh1gvrk/parquet/artists')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# get filepath to log data file
INPUT_DATA = 's3a://udacity-dend/log_data'

log_data = INPUT_DATA + "/2018/*/*.json"
log_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://udacity-dend/log_data/2018/*/*.json'

In [28]:
# read log data file
df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# filter by actions for song plays
df = df.filter(df.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# extract columns for users table    
users_table = df.select(['userId', 'firstName', 'lastName', 'gender', 'level']).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet('s3a://desh1gvrk/parquet/users')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000.0),TimestampType())
df = df.withColumn("timestamp", get_timestamp(df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000.0), DateType())
df = df.withColumn("datetime", get_datetime(df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# extract columns to create time table
time_table = df.select(col('ts').alias('start_time'),
                    f.hour(df.timestamp).alias('hour'),
                    f.dayofweek(df.datetime).alias('day'),
                    f.weekofyear(df.datetime).alias('week'),
                    f.month(df.datetime).alias('month'), 
                    f.year(df.datetime).alias('year'),
                    date_format(df.datetime, 'E').alias('weekday')).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# write time table to parquet files partitioned by year and month
time_table.write.mode('overwrite').partitionBy('year', 'month').parquet('s3a://desh1gvrk/parquet/time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# read in song data to use for songplays table
INPUT_DATA = 's3a://udacity-dend/song_data' #config['S3']['SONG_DATA']
song_data = INPUT_DATA + "/A/A/A/*.json"

song_df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# extract columns from joined song and log datasets to create songplays table 
songplays = df.join(song_df,
        (df.song == song_df.title) & (df.artist == song_df.artist_name) & (df.length == song_df.duration), how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
songplays_table = songplays.select(df.ts.alias('start_time'), f.year(df.datetime).alias('year'), f.month(df.datetime).alias('month'), 
                                   df.userId.alias('user_id'), df.level, song_df.song_id, song_df.artist_id, df.sessionId.alias('session_id'), df.location, 
                                   df.userAgent.alias('user_agent'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.mode('overwrite').partitionBy('year', 'month').parquet('s3a://desh1gvrk/parquet/songplays')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…